### Balance the data

In [8]:
import numpy as np
import pandas as pd
from collections import Counter
from random import shuffle

train_data = np.load('training_data.npy')

df = pd.DataFrame(train_data)
print(df.head())
print(Counter(df[1].apply(str)))

lefts = []
rights = []
forwards = []

shuffle(train_data)

for data in train_data:
    img = data[0]
    choice = data[1]
    if choice == [0,0,1,0]:
        lefts.append([img,choice])
    elif choice == [1,0,0,0]:
        forwards.append([img,choice])
    elif choice == [0,0,0,1]:
        rights.append([img,choice])
    else:
        print('no matches')


forwards = forwards[:len(lefts)][:len(rights)]
lefts = lefts[:len(forwards)]
rights = rights[:len(forwards)]

final_data = forwards + lefts + rights
shuffle(final_data)
print(lefts)
print(forwards)
print(rights)
np.save('training_data.npy', final_data)

                                                   0             1
0  [[43, 43, 43, 43, 43, 43, 43, 50, 31, 34, 39, ...  [1, 1, 1, 1]
1  [[43, 43, 43, 43, 43, 43, 43, 50, 31, 34, 39, ...  [1, 0, 1, 0]
2  [[43, 43, 43, 43, 43, 43, 43, 50, 31, 34, 39, ...  [0, 0, 0, 0]
3  [[43, 43, 43, 43, 43, 43, 43, 50, 31, 34, 39, ...  [0, 1, 0, 1]
4  [[43, 43, 43, 43, 43, 43, 43, 50, 31, 34, 39, ...  [0, 1, 1, 0]
Counter({'[0, 0, 0, 0]': 34, '[1, 0, 0, 0]': 22, '[0, 0, 1, 0]': 22, '[0, 0, 0, 1]': 16, '[0, 1, 0, 0]': 12, '[1, 0, 1, 0]': 2, '[0, 1, 0, 1]': 2, '[0, 1, 1, 0]': 2, '[0, 0, 1, 1]': 2, '[1, 1, 1, 1]': 1, '[1, 0, 0, 1]': 1})
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches


## Let's create some models!

### AlexNet

In [9]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization

def alexnet(width, height, lr):
    network = input_data(shape=[None, width, height, 1], name='input')
    network = conv_2d(network, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 3, activation='softmax')
    network = regression(network, optimizer='momentum',
                         loss='categorical_crossentropy',
                         learning_rate=lr, name='targets')

    model = tflearn.DNN(network, checkpoint_path='model_alexnet',
                        max_checkpoints=1, tensorboard_verbose=2, tensorboard_dir='log')


    return model

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


## TRAINING

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
from random import shuffle

WIDTH = 227
HEIGHT = 227
LR = 1e-3
EPOCHS = 10
MODEL_NAME = 'f1-car-{}-{}-{}-epochs-300K-data.model'.format(LR, 'alexnetv2',EPOCHS)

model = alexnet(WIDTH, HEIGHT, LR)

train_data = np.load('training_data.npy')
print(i)
train = train_data[:-100]
test = train_data[-100:]

X = np.array([i[0] for i in train]).reshape(-1,WIDTH,HEIGHT,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,WIDTH,HEIGHT,1)
test_y = [i[1] for i in test]
print(test_x)

model.fit({'input': X}, {'targets': Y}, n_epoch=10, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

#     model.fit(X, Y, n_epoch=1, validation_set=({'input': test_x}, {'targets': test_y}), shuffle=True,
#           show_metric=True, snapshot_step=200, run_id=MODEL_NAME)
model.save(MODEL_NAME)



# tensorboard --logdir=foo:C:/path/to/log